# Personalized Medicine: Redefining Cancer Treatment#
## Predict the effect of Genetic Variants to enable Personalized Medicine

A lot has been said during the past several years about how precision medicine and, more concretely, how genetic testing is going to disrupt the way diseases like cancer are treated.

Once sequenced, a cancer tumor can have thousands of genetic mutations. But the challenge is distinguishing the mutations that contribute to tumor growth (drivers) from the neutral mutations (passengers). 

Currently this interpretation of genetic mutations is being done manually. This is a very time-consuming task where a clinical pathologist has to manually review and classify every single genetic mutation based on evidence from text-based clinical literature.

For this competition MSKCC is making available an expert-annotated knowledge base where world-class researchers and oncologists have manually annotated thousands of mutations.

We will develop a Machine Learning algorithm that, using this knowledge base as a baseline, automatically classifies genetic variations.

# Data Set

In this project you will develop algorithms to classify genetic mutations based on clinical evidence (text).
There are nine different classes a genetic mutation can be classified on.

This is not a trivial task since interpreting clinical evidence is very challenging even for human specialists. Therefore, modeling the clinical evidence (text) will be critical for the success of your approach.

Both, training and test, data sets are provided via two different files. One (training/test_variants) provides the information about the genetic mutations, whereas the other (training/test_text) provides the clinical evidence (text) that our human experts used to classify the genetic mutations. Both are linked via the ID field.

Therefore the genetic mutation (row) with ID=15 in the file training_variants, was classified using the clinical evidence (text) from the row with ID=15 in the file training_text


# Import Required Libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import *
import sklearn
import xgboost as xgb

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/msk-redefining-cancer-treatment/stage2_sample_submission.csv.7z
/kaggle/input/msk-redefining-cancer-treatment/stage_2_private_solution.csv.7z
/kaggle/input/msk-redefining-cancer-treatment/stage1_solution_filtered.csv.7z
/kaggle/input/msk-redefining-cancer-treatment/stage2_test_variants.csv.7z
/kaggle/input/msk-redefining-cancer-treatment/stage2_test_text.csv.7z
/kaggle/input/msk-redefining-cancer-treatment/test_text.zip
/kaggle/input/msk-redefining-cancer-treatment/test_variants.zip
/kaggle/input/msk-redefining-cancer-treatment/training_text.zip
/kaggle/input/msk-redefining-cancer-treatment/training_variants.zip


# Load the data Set

In [2]:
train = pd.read_csv('../input/msk-redefining-cancer-treatment/training_variants.zip', compression='zip', header=0, sep=',', quotechar='"')
test = pd.read_csv('../input/msk-redefining-cancer-treatment/test_variants.zip', compression='zip', header=0, sep=',', quotechar='"')

trainx = pd.read_csv('../input/msk-redefining-cancer-treatment/training_text.zip',compression='zip', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
testx = pd.read_csv('../input/msk-redefining-cancer-treatment/test_text.zip',compression='zip', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])

# Exploratory Data Analysis

In [3]:
train.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 4 columns):
ID           3321 non-null int64
Gene         3321 non-null object
Variation    3321 non-null object
Class        3321 non-null int64
dtypes: int64(2), object(2)
memory usage: 103.9+ KB


In [5]:
test.head()

,ID,Gene,Variation
0,0,ACSL4,R570S
1,1,NAGLU,P521L
2,2,PAH,L333F
3,3,ING1,A148D
4,4,TMEM216,G77A


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5668 entries, 0 to 5667
Data columns (total 3 columns):
ID           5668 non-null int64
Gene         5668 non-null object
Variation    5668 non-null object
dtypes: int64(1), object(2)
memory usage: 133.0+ KB


In [7]:
trainx.head()

,ID,Text
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [8]:
trainx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 2 columns):
ID      3321 non-null int64
Text    3316 non-null object
dtypes: int64(1), object(1)
memory usage: 52.0+ KB


In [9]:
testx.head()

,ID,Text
0,0,2. This mutation resulted in a myeloproliferat...
1,1,Abstract The Large Tumor Suppressor 1 (LATS1)...
2,2,Vascular endothelial growth factor receptor (V...
3,3,Inflammatory myofibroblastic tumor (IMT) is a ...
4,4,Abstract Retinoblastoma is a pediatric retina...


In [10]:
testx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5668 entries, 0 to 5667
Data columns (total 2 columns):
ID      5668 non-null int64
Text    5667 non-null object
dtypes: int64(1), object(1)
memory usage: 88.7+ KB


# Create Features and Target Variable

We will merge train dataset with train text dataset. Also we will manage NA columns with pandas fillna().
Then we will assign "Class" feature to target variable and we will drop it from the train data set.

In [11]:
train = pd.merge(train, trainx, how='left', on='ID').fillna('')
y = train['Class'].values
train = train.drop(['Class'], axis=1)

We will merge both test data set and test text data as above. Any way test data set is not containing "Class" feature. 

In [12]:
test = pd.merge(test, testx, how='left', on='ID').fillna('')
pid = test['ID'].values

# Feature Engineering

In [13]:
df_all = pd.concat((train, test), axis=0, ignore_index=True)
df_all['Gene_Share'] = df_all.apply(lambda r: sum([1 for w in r['Gene'].split(' ') if w in r['Text'].split(' ')]), axis=1)
df_all['Variation_Share'] = df_all.apply(lambda r: sum([1 for w in r['Variation'].split(' ') if w in r['Text'].split(' ')]), axis=1)

# Dataset Pre Processing

In [14]:
for c in df_all.columns:
    if df_all[c].dtype == 'object':
        if c in ['Gene','Variation']:
            lbl = preprocessing.LabelEncoder()
            df_all[c+'_lbl_enc'] = lbl.fit_transform(df_all[c].values)  
            df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
            df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' ')))
        elif c != 'Text':
            lbl = preprocessing.LabelEncoder()
            df_all[c] = lbl.fit_transform(df_all[c].values)
        if c=='Text': 
            df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
            df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' '))) 

train = df_all.iloc[:len(train)]
test = df_all.iloc[len(train):]

In [15]:
class cust_regression_vals(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        x = x.drop(['Gene', 'Variation','ID','Text'],axis=1).values
        return x

class cust_txt_col(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        return x[self.key].apply(str)

In [16]:
print('Pipeline...')
fp = pipeline.Pipeline([
    ('union', pipeline.FeatureUnion(
        n_jobs = -1,
        transformer_list = [
            ('standard', cust_regression_vals()),
            ('pi1', pipeline.Pipeline([('Gene', cust_txt_col('Gene')), ('count_Gene', feature_extraction.text.CountVectorizer(analyzer=u'char', ngram_range=(1, 8))), ('tsvd1', decomposition.TruncatedSVD(n_components=20, n_iter=25, random_state=12))])),
            ('pi2', pipeline.Pipeline([('Variation', cust_txt_col('Variation')), ('count_Variation', feature_extraction.text.CountVectorizer(analyzer=u'char', ngram_range=(1, 8))), ('tsvd2', decomposition.TruncatedSVD(n_components=20, n_iter=25, random_state=12))])),
            
        ])
    )])

Pipeline...


In [17]:
train = fp.fit_transform(train); print(train.shape)
test = fp.transform(test); print(test.shape)
y = y - 1 #fix for zero bound array

(3321, 50)
(5668, 50)


# XG Boost Model Training

In [18]:
denom = 0
fold = 1 #Change to 5, 1 for Kaggle Limits
for i in range(fold):
    params = {
        'eta': 0.03333,
        'max_depth': 4,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 9,
        'seed': i,
        'silent': True
    }
    x1, x2, y1, y2 = model_selection.train_test_split(train, y, test_size=0.18, random_state=i)
    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
    model = xgb.train(params, xgb.DMatrix(x1, y1), 1000,  watchlist, verbose_eval=50, early_stopping_rounds=100)
    score1 = metrics.log_loss(y2, model.predict(xgb.DMatrix(x2), ntree_limit=model.best_ntree_limit), labels = list(range(9)))
    print(score1)
    #if score < 0.9:
    if denom != 0:
        pred = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit+80)
        preds += pred
    else:
        pred = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit+80)
        preds = pred.copy()
    denom += 1
    submission = pd.DataFrame(pred, columns=['class'+str(c+1) for c in range(9)])
    submission['ID'] = pid
    submission.to_csv('submission_xgb_fold_'  + str(i) + '.csv', index=False)
preds /= denom
submission = pd.DataFrame(preds, columns=['class'+str(c+1) for c in range(9)])
submission['ID'] = pid
submission.to_csv('submission_xgb.csv', index=False)

[0]	train-mlogloss:2.14625	valid-mlogloss:2.15179
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[50]	train-mlogloss:1.22697	valid-mlogloss:1.37848
[100]	train-mlogloss:0.96216	valid-mlogloss:1.20367
[150]	train-mlogloss:0.81131	valid-mlogloss:1.13104
[200]	train-mlogloss:0.70775	valid-mlogloss:1.09930
[250]	train-mlogloss:0.62814	valid-mlogloss:1.07859
[300]	train-mlogloss:0.56471	valid-mlogloss:1.06977
[350]	train-mlogloss:0.51138	valid-mlogloss:1.06579
[400]	train-mlogloss:0.46662	valid-mlogloss:1.06384
[450]	train-mlogloss:0.42542	valid-mlogloss:1.06362
[500]	train-mlogloss:0.38576	valid-mlogloss:1.06294
Stopping. Best iteration:
[425]	train-mlogloss:0.44648	valid-mlogloss:1.06274

1.0627418830445703


In [19]:
submission.head()

,class1,class2,class3,class4,class5,class6,class7,class8,class9,ID
0,0.253041,0.054113,0.361450,0.065566,0.066544,0.009476,0.175423,0.005165,0.009222,0
1,0.203772,0.088146,0.009943,0.311965,0.031091,0.017503,0.331196,0.000707,0.005678,1
2,0.228855,0.280346,0.003842,0.243845,0.064525,0.010433,0.166777,0.000984,0.000393,2
3,0.229815,0.068399,0.010867,0.289495,0.071386,0.026421,0.287756,0.000638,0.015222,3
4,0.265078,0.175754,0.006064,0.237577,0.098730,0.188054,0.024646,0.003120,0.000976,4


**We can see from the submission table that probability for each of nine mutations are classfied for each sample.**

**So we can identify whcih mutation caused the cancer for each patient. This will enable doctors to treat patients more fast and accurate.**

## Further work

**More accurate model can be trained with further feature engineering and deeplearning models.....**